In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.kill(os.getpid(), 9)

In [2]:
!pip install -q langchain==0.3.0 langchain-community==0.3.0 langchain-core==0.3.0 langchain-chroma langchain-text-splitters


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 118.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 kB 10.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.57.6 requires tokenizers<=0.23.0,>=0.22.0, but you have tokenizers 0.20.3 which is incompatible.
langchain-classic 1.0.1 requires langchain-core<2.0.0,>=1.2.5, but you have langchain-core 0.3.0 which is incompatible.
langchain-classic 1.0.1 requires langchain-text-splitters<2.0.0,>=1.1.0, but you have langchain-text-splitters 0.3.0

In [ ]:
import os

from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, NewsURLLoader
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.document_loaders import TextLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


In [ ]:
import os
os.kill(os.getpid(), 9)


In [ ]:
with open(r"/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/CNN_Links.txt") as f:
    urls = [link.replace("\n", "") for link in set(f.readlines())]

loader = NewsURLLoader(urls=urls)
data = loader.load()
print("Primeira notícia: ", data[0])
print("\nSegunda notícia: ", data[1])


In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

#vectordb = Chroma.from_texts([], embedding=embedding, persist_directory=persist_directory)
contents = [item.page_content for item in data]
sources = [{'source_info': item.metadata['link']} for item in data]

# Initialize Chroma without specifying texts initially
vectordb = Chroma.from_texts(contents, embedding, metadatas = sources, persist_directory=persist_directory)

vectordb.persist()

In [ ]:
#Declaração do modelo de embedding
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'
embedding = instructor_embeddings

#Criação do objeto de gestão da vector db
vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory)

retriever = vectordb.as_retriever()
query = input("The first question:  ")
#Retorna os 3 primeiros itens relevantes para a pergunta do usuário
context = retriever.get_relevant_documents(query)[:1]

In [ ]:
context